In [21]:
pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 304.8 kB/s eta 0:00:25
   -- ------------------------------------- 0.5/8.1 MB 304.8 kB/s eta 0:00:25
   -- ------------------------------------- 0.5/8.1 MB 304.8 kB/s eta 0:00:25
   -- ------------------------------------- 0.5/8.1 MB 304.8 kB/s eta 0:00:25
   

In [ ]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
#!pip3 install pandas
#!pip install albumentations

In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import os


print("Execution status: all import statements successful")

# -------------------------------
# 0. Setup and Paths
# -------------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()
print("Execution status: seeding completed")

# Update these paths to your local system
image_dir = r"D:\Project_Space\5.retina\retina_data\B. Disease Grading\1. Original Images\a. Training Set"
label_csv = r"D:\Project_Space\5.retina\retina_data\B. Disease Grading\2. Groundtruths\a. IDRiD_Disease Grading_Training Labels.csv"
mask_dirs = [
    r"D:\Project_Space\5.retina\retina_data\A. Segmentation\2. All Segmentation Groundtruths\a. Training Set\1. Microaneurysms",
    r"D:\Project_Space\5.retina\retina_data\A. Segmentation\2. All Segmentation Groundtruths\a. Training Set\2. Haemorrhages",
    r"D:\Project_Space\5.retina\retina_data\A. Segmentation\2. All Segmentation Groundtruths\a. Training Set\3. Hard Exudates",
    r"D:\Project_Space\5.retina\retina_data\A. Segmentation\2. All Segmentation Groundtruths\a. Training Set\4. Soft Exudates",
    r"D:\Project_Space\5.retina\retina_data\A. Segmentation\2. All Segmentation Groundtruths\a. Training Set\5. Optic Disc"
]

image_paths = sorted(glob.glob(image_dir + r"\*.jpg"))
label_df = pd.read_csv(label_csv)
filename_to_label = dict(zip(label_df['Image name'], label_df['Retinopathy grade']))
labels = [filename_to_label[os.path.splitext(os.path.basename(p))[0]] for p in image_paths]

print("Execution status: defining dataset class")


























def save_visualization(image_tensor, pred_mask, true_mask, epoch, index):
    image = image_tensor.cpu().permute(1, 2, 0).numpy()
    pred = torch.sigmoid(pred_mask).cpu().numpy()
    true = true_mask.cpu().numpy()

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(image)
    axs[0].set_title("Original Image")
    axs[1].imshow(pred[0], cmap='Reds')
    axs[1].set_title("Predicted Mask")
    axs[2].imshow(true[0], cmap='Greens')
    axs[2].set_title("Ground Truth Mask")
    for ax in axs: ax.axis('off')

    os.makedirs("results", exist_ok=True)
    plt.savefig(f"results/epoch_{epoch}_sample_{index}.png")
    plt.close()



























# -------------------------------
# 1. Dataset Class
# -------------------------------
class IDRiDDataset(Dataset):
    def __init__(self, image_paths, labels, mask_dirs, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.mask_dirs = mask_dirs
        self.mask_suffixes = ['_MA', '_HE', '_EX', '_SE', '_OD']
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        base_name = os.path.splitext(os.path.basename(self.image_paths[idx]))[0]

        masks = []
        for mask_dir, suffix in zip(self.mask_dirs, self.mask_suffixes):
            mask_path = os.path.join(mask_dir, base_name + suffix + ".tif")
            mask = Image.open(mask_path).convert("L")
            masks.append(np.array(mask))

        mask_stack = np.stack(masks, axis=0)  # Shape: (5, H, W)

        if self.transform:
            augmented = self.transform(image=np.array(image), mask=mask_stack.transpose(1, 2, 0))
            image = augmented['image']
            mask_stack = augmented['mask'].permute(2, 0, 1)  # Back to (5, H, W)

        return image, torch.tensor(label, dtype=torch.long), mask_stack.float()

print("Execution status: transformations")

# -------------------------------
# 2. Transformations
# -------------------------------
transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

print("Execution status: model definitions")

# -------------------------------
# 3. Model Definition
# -------------------------------
class SharedBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])  # Remove avgpool and fc

    def forward(self, x):
        return self.backbone(x)  # Output shape: [B, 512, H/32, W/32]

class ClassificationExpert(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(in_channels, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)


class SegmentationExpert(nn.Module):
    def __init__(self, in_channels=512, out_channels=5):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels, 256, kernel_size=4, stride=2, padding=1),  # 8 → 16
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),          # 16 → 32
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),           # 32 → 64
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),            # 64 → 128
            nn.ReLU(),
            nn.ConvTranspose2d(32, out_channels, kernel_size=4, stride=2, padding=1),  # 128 → 256
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)


class MultiTaskModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.shared = SharedBackbone()
        self.classifier = ClassificationExpert(512, num_classes)
        self.segmenter = SegmentationExpert()

    def forward(self, x):
        features = self.shared(x)
        class_out = self.classifier(features)
        seg_out = self.segmenter(features)
        return class_out, seg_out

print("Execution status: defining loss function")

# -------------------------------
# 4. Loss Function
# -------------------------------
def multitask_loss(class_pred, class_target, seg_pred, seg_target, alpha=0.5):
    classification_loss = nn.CrossEntropyLoss()(class_pred, class_target)
    segmentation_loss = nn.BCEWithLogitsLoss()(seg_pred, seg_target)
    return alpha * classification_loss + (1 - alpha) * segmentation_loss

print("Execution status: defining training and evaluation")

# -------------------------------
# 5. Training and Evaluation
# -------------------------------
def train(model, dataloader, optimizer, device):
    with open("training_log.txt", "a") as f:
        f.write("f{epoch},{batch_idx},{loss.item():.4f}\n")
    model.train()
    with open("training_log.csv", "w") as f:
            f.write("epoch,batch,loss\n")
    for images, labels, masks in dataloader:
        images, labels, masks = images.to(device), labels.to(device), masks.to(device)
        optimizer.zero_grad()
        class_out, seg_out = model(images)
        loss = multitask_loss(class_out, labels, seg_out, masks)
        loss.backward()
        optimizer.step()

def evaluate(model, dataloader, device):
    model.eval()
    total_correct = 0
    total_samples = 0
    dice_scores = []
    if batch_idx == 0:
        for i in range(min(2, images.size(0))):
            save_visualization(images[i], seg_out[i], masks[i], epoch, i)

    with torch.no_grad():
        for images, labels, masks in dataloader:
            images, labels, masks = images.to(device), labels.to(device), masks.to(device)
            class_out, seg_out = model(images)
            preds = torch.argmax(class_out, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)

            seg_out = torch.sigmoid(seg_out)
            intersection = (seg_out * masks).sum(dim=(2, 3))
            union = seg_out.sum(dim=(2, 3)) + masks.sum(dim=(2, 3))
            dice = (2. * intersection) / (union + 1e-8)
            dice_scores.extend(dice.mean(dim=1).cpu().numpy())

    accuracy = total_correct / total_samples
    avg_dice = sum(dice_scores) / len(dice_scores)
    return accuracy, avg_dice

print("Execution status: main function")



# -------------------------------
# 6. Main Execution
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = MultiTaskModel(num_classes=5).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

dataset = IDRiDDataset(image_paths, labels, mask_dirs, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

num_epochs = 3
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train(model, dataloader, optimizer, device)
    acc, dice = evaluate(model, dataloader, device)
    print(f"Accuracy: {acc:.4f}, Dice Score: {dice:.4f}")


Execution status: all import statements successful
Execution status: seeding completed
Execution status: defining dataset class
Execution status: transformations
Execution status: model definitions
Execution status: defining loss function
Execution status: defining training and evaluation
Execution status: main function
Using device: cuda


d:\Project_Space\5.retina\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Project_Space\5.retina\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/3


NameError: name 'batch_idx' is not defined